# Modification adfter meeting 

1. delete theta, phi, mu, only keep U, g, r. In addition, add error range for flow rate (fr)
2. rewrite class function for more general application
3. less initial datapoints, increase iterations

In [1]:
import numpy as np
import pandas as pd
from sklearn.gaussian_process import GaussianProcessRegressor
import math
from scipy.stats import norm
from scipy.optimize import minimize 
import sys

if not sys.warnoptions:
    import warnings
    warnings.simplefilter("ignore")

In [18]:
# newtonian list add theta, phi, mu
property_list_newtonian = [30,1,0.045,0.066,1210]
# non_newtonian property_list = [0.53,0.25,0.065,1143.5]
x_f = 0.0005
x_d = 0.101
h_w = 0.1
quantile = 0.5

In [62]:
def objective(x_input,ink_type, ink_value,judge = x_f, qt = quantile,Xd = x_d):
    """
    objective function to max rewards 
    
    design this function with quantile weight, inspired by quantle loss regression idea and rewards matrix used in the RL notebook

    for slot die coating, two edge cases are: leaking, right: breaking-up

    leakings are zero-tolerance defects. goal is to encourage more prediction points on right side

    R(x_u,x_f) = (x_f - x_u)*tao if x_u - half point > 0;

               = (x_u - 0)*(1-tao) if x_u - half point < 0 
    
    interpretation : if tao = 0.1, then R will be larger for points located on left side, which potentially increase chance to make x_u on left part of middle point
    goal is to max rewards 
    """
    # get output from input, input is a matrix with 6 columns
    # get ink properties:
    mu,n,m,sigma,density = ink_value
    y = []
    x_location = []
    if ink_type == "Non_Newtonian":
        for x_i in x_input: 
        # searching values and doing calculations
            U, g, r, fr = x_i  
            # delete theta, phi, mu. only keep U, g, r. In addition, add error range for fr
            thickness = fr/(U*h_w) 

            a = (U*(n+1)*(2*n+1)/n)**n*(g**(-n-1))
            b_s = (U*(g-2*thickness)*(n+1)*(2*n+1)/n)**n * (g**(-2*n-1))
            b_l = -(U*(2*thickness-g)*(n+1)*(2*n+1)/n)**n * (g**(-2*n-1))
            modified_CA = ((U*m*(U/g)**(n-1))/sigma)**(2/3)      
            p_ambient = density*U**2/2

            if thickness <= g/2:
                xu_estimate = judge - (p_ambient - 1.34*modified_CA*sigma/thickness - (Xd-judge)*m*b_s - sigma*(math.cos(theta)+math.cos(phi)))/a*m

            elif thickness > g/2:
                xu_estimate = judge - (p_ambient - 1.34*modified_CA*sigma/thickness - (Xd-judge)*m*b_l - sigma*(math.cos(theta)+math.cos(phi)))/a*m
            
            x_location.append(xu_estimate)
            if xu_estimate <= 0:
                y.append(xu_estimate*2)
            elif xu_estimate >= judge/2 and xu_estimate < judge:
                y.append((judge - xu_estimate)*qt)
            elif xu_estimate < judge/2 and xu_estimate > 0:
                y.append(xu_estimate*(1-qt))
            else:
                y.append(-xu_estimate*2)
        
    elif ink_type == "Newtonian":
        for x_i in x_input: 
        # searching values and doing calculations
            U, g, r, fr = x_i  
            # delete theta, phi, mu. only keep U g r. In addition, add error range for fr
            thickness = fr/(U*h_w) 

            a = (U*(n+1)*(2*n+1)/n)**n*(g**(-n-1))
            b_s = (U*(g-2*thickness)*(n+1)*(2*n+1)/n)**n * (g**(-2*n-1))
            b_l = -(U*(2*thickness-g)*(n+1)*(2*n+1)/n)**n * (g**(-2*n-1))
            modified_CA = ((U*m*(U/g)**(n-1))/sigma)**(2/3)      
            p_ambient = density*U**2/2

            if thickness <= g/2:
                xu_estimate = judge - g**2*(p_ambient - 1.34*modified_CA*sigma/thickness - (Xd-judge)*m*b_s - sigma*(math.cos(theta)+math.cos(phi))/g)/(6*mu*U)
                     
            elif thickness > g/2:
                xu_estimate = judge - g**2*(p_ambient - 1.34*modified_CA*sigma/thickness - (Xd-judge)*m*b_l - sigma*(math.cos(theta)+math.cos(phi))/g)/(6*mu*U)
            
            x_location.append(xu_estimate)
            if xu_estimate <= 0:
                y.append(xu_estimate*2)
            elif xu_estimate >= judge/2 and xu_estimate < judge:
                y.append((judge - xu_estimate)*qt)
            elif xu_estimate < judge/2 and xu_estimate > 0:
                y.append(xu_estimate*(1-qt))
            else:
                y.append(-xu_estimate*2)
    return  y  #,x_location uncommend this if you want to check x_u location
        

In [23]:
def initial_data():
#     theta = np.random.choice(np.arange(120,140),self.batch_size)  # 10 扩大范围 看看有没有奇怪的suggestion
#     phi = np.random.choice(np.arange(60,80),self.batch_size)  # 100 需要有一个相互依靠的关系
    # U, g, r, fr = x_i
    U = np.random.uniform(low = 0.05, high = 0.5, size = 100)
    g = np.random.choice(np.arange(250,500)*1e-6, size = 100)
    r = np.random.uniform(low = 1.5, high = 4, size = 100)
    fr = np.random.uniform(low = 0.05, high = 0.2, size = 100)
    input_matrix = np.stack([U, g, r, fr], axis=1)
    return input_matrix

In [40]:
class Bayesian_opt():
    def __init__(self, target_func,ink_type, ink_value, x_init, y_init, n_iter, batch_size, input_list):
        # initial values
        self.x_init = x_init
        self.y_init = y_init
        self.n_iter = n_iter
        self.target_func = target_func
        self.batch_size = batch_size  
        self.input_list = input_list
        self.ink_type = ink_type
        self.ink_value = ink_value
        self.best_samples = pd.DataFrame(columns = (self.input_list + ["y","ei"]))
        self.gauss_pr = GaussianProcessRegressor()
        self.distances_ = []
    # surrogate function using EI method, as a starting point  
    def _get_expected_improvement(self, x_new): 
        # do more research on different methods and figure out why choose EI
        # Using estimate from Gaussian surrogate instead of actual function for a new trial data point to avoid cost 
        mean_y_new, sigma_y_new = self.gauss_pr.predict(np.array([x_new]), return_std=True)
        sigma_y_new = sigma_y_new.reshape(-1,1)
        if sigma_y_new < 0.0:
            return 0.0
        
        # Using estimates from Gaussian surrogate instead of actual function for entire prior distribution to avoid cost
        # add a very small to avoid divide by zero sigma 
        mean_y = self.gauss_pr.predict(self.x_init)
        max_mean_y = np.max(mean_y)
        z = (mean_y_new - max_mean_y) / (sigma_y_new+1e-9)
        exp_imp = (mean_y_new - max_mean_y) * norm.cdf(z) + sigma_y_new * norm.pdf(z)
        
        return exp_imp
    
    def acquisition_function(self,x):
        # max (function) = min (-function)
        return -self._get_expected_improvement(x)
    
    ###### modify this
    def random_select_inputs(self):
#         theta = np.random.choice(np.arange(120,140),self.batch_size)  # 10 扩大范围 看看有没有奇怪的suggestion
#         phi = np.random.choice(np.arange(60,80),self.batch_size)  # 100 需要有一个相互依靠的关系
        # start with less points and larger iterations
        U = np.random.uniform(low = 0.05, high = 1.5, size = self.batch_size)
        g = np.random.choice(np.arange(250,500)*1e-6, size = self.batch_size)
        r = np.random.uniform(low = 1.5, high = 4, size = self.batch_size)
        fr = np.random.uniform(low = 0.05, high = 1, size = self.batch_size)
        random_matrix = np.stack([U, g, r, fr], axis=1)
        return random_matrix

    def _get_next_probable_point(self):
        
        min_ei = float(sys.maxsize)
        x_optimal = None 
        ###### fix bounds 
        bounds = [(1e-6, 1.6), (1e-6, 500*1e-6), (1e-6, 4.1),(1e-6, 1.1)]
        # Trial with an array of input_simulated_data
        random_matrix = self.random_select_inputs()
        for x_start in random_matrix:
            response = minimize(fun=self.acquisition_function, x0=x_start, method='L-BFGS-B',bounds = bounds)
            if response.fun < min_ei:
                min_ei = response.fun
                x_optimal = response.x

        return x_optimal, min_ei  
        
    # wait for debug for rest of two functions     
    def _extend_prior_with_posterior_data(self, x,y):
        
        self.x_init = np.append(self.x_init, np.array([x]), axis = 0)
        self.y_init = np.append(self.y_init, np.array([y]), axis = 0)
  
    def optimize(self):
        y_max_ind = np.argmax(self.y_init)
        y_max = self.y_init[y_max_ind]
        optimal_x = self.x_init[y_max_ind]
        optimal_ei = None
        for i in range(self.n_iter):
            self.gauss_pr.fit(self.x_init, self.y_init)
            x_next, ei = self._get_next_probable_point()
            y_next = self.target_func(np.array([x_next]),self.ink_type,self.ink_value)[0]
            self._extend_prior_with_posterior_data(x_next,y_next)
            # 
            if y_next > y_max:
                y_max = y_next
                optimal_x = x_next
                optimal_ei = ei

            if i == 0:
                 prev_x = x_next
            else:
                self.distances_.append(np.linalg.norm(prev_x - x_next))
                prev_x = x_next
            
            self.best_samples = self.best_samples.append({"Web_speed(m/s)":optimal_x[0],"coating_height(m)":optimal_x[1],"ratio":optimal_x[2],
                                                           "Flow_rate(m/s)":optimal_x[3],"y": y_max, "ei": optimal_ei},ignore_index=True)
        
        return optimal_x, y_max 
    

In [58]:
theta = np.random.choice(np.arange(10,140),size =1)
phi = np.random.choice(np.arange(60,100),size =1)
init_X = initial_data()
init_Y = objective(init_X,ink_type = "Newtonian", ink_value = property_list_newtonian)

In [61]:
max(init_Y)

-1.5552154610249296

In [59]:
print(theta,phi)

[107] [62]


In [60]:
# bayesian for newtonian
bopt_newtonian = Bayesian_opt(target_func=objective, 
                              ink_type = "Newtonian",
                              ink_value = property_list_newtonian,
                              x_init=init_X,
                              y_init=init_Y, 
                              n_iter=300,
                              batch_size=30,
                              input_list =["Web_speed(m/s)","coating_height(um)","ratio","Flow_rate(m/s)"])
bopt_newtonian.optimize()

(array([2.53573884e-02, 5.00000000e-04, 1.00000000e-06, 1.00000000e-06]),
 3.7137806739023966e-05)

In [63]:
objective(x_input = [[2.53573884e-02, 5.00000000e-04, 1.00000000e-06, 1.00000000e-06]],
          ink_type = "Newtonian", ink_value =property_list_newtonian)

([3.7137806563719154e-05], [0.0004257243868725617])